# The General Claims Data
- Primary question of interest: Who was at fault?
<br> <br>
- Secondary questions of interest:
    - What Claim Group should the claim be routed to?
    - Can we predict severity type (minor, moderate, major, life threatening, death)
    - What was the cause of the loss?

In [12]:
import pandas as pd
import numpy as np
import spacy
import en_core_web_lg

In [25]:
import nltk
from nltk import word_tokenize

In [37]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [5]:
import gensim

In [2]:
general_df = pd.read_csv('data/NSS_DS_data.thegeneral.csv')

In [3]:
general_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,LossCauseName,CCCreateTime,CloseDate
0,10000,130658,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,left side damages towed by unknown none front...,Moderate,Field Ops 1,Other party at fault,Collision with motor vehicle,2015-03-12 09:05:17.9100000,2015-04-29 11:25:18.3190000
1,10001,130659,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Field Ops 1,Other party at fault,Collision while merging / lane change,2015-03-12 11:46:23.1590000,2015-07-06 08:26:47.5350000
2,10002,130660,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Field Ops 1,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000
3,10003,130661,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Field Ops 1,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000
4,10004,130662,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,Collision with motor vehicle,2015-03-12 16:35:17.3570000,2016-01-29 13:54:25.6150000


In [6]:
general_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120657 entries, 0 to 120656
Data columns (total 13 columns):
ClaimID_RGEN           120657 non-null int64
ExposureID_RGEN        120657 non-null int64
ClaimLevelBody         120657 non-null object
ExposureLevelBody      111166 non-null object
AccidentDescription    120657 non-null object
InjuryDescription      120619 non-null object
DamageDescription      120657 non-null object
SeverityTypeName       87646 non-null object
ClaimGroup             120642 non-null object
FaultRatingName        120656 non-null object
LossCauseName          120657 non-null object
CCCreateTime           120657 non-null object
CloseDate              99130 non-null object
dtypes: int64(2), object(11)
memory usage: 12.0+ MB


In [13]:
nlp = en_core_web_lg.load()

In [14]:
def column_to_list_text(df, col_name):
    return df[col_name].tolist()

In [32]:
def lower_columns(df, col_list):
    n_df = df
    for col_name in col_list:
        n_col = col_name + '_preprocess'
        n_df[n_col] = n_df[col_name].str.lower()
    return n_df

In [33]:
lowered_df = lower_columns(general_df, ['ClaimLevelBody', 'ExposureLevelBody', 'AccidentDescription', 
                                        'InjuryDescription', 'DamageDescription'])

In [48]:
def tokenize(row, col_name):
#     list_cols = [['ClaimLevelBody_preprocess', 'ExposureLevelBody_preprocess', 'AccidentDescription_preprocess', 
#                                         'InjuryDescription_preprocess', 'DamageDescription_preprocess']]
    text = row[col_name]
    text_tokenize = [token for token in word_tokenize(text) if token.isalpha() and token not in stop]
    return text_tokenize
        
def pre_process(df, col_list):
    for col in col_list:
        df[col] = df.apply(lambda x: tokenize(x, col), axis=1)

In [49]:
out_df = pre_process(lowered_df, ['ClaimLevelBody_preprocess', 'ExposureLevelBody_preprocess', 
                                   'AccidentDescription_preprocess', 
                                   'InjuryDescription_preprocess', 'DamageDescription_preprocess'])

KeyboardInterrupt: 

In [ ]:
out_df.head()

In [34]:
lowered_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,LossCauseName,CCCreateTime,CloseDate,n_col,ClaimLevelBody_preprocess,ExposureLevelBody_preprocess,AccidentDescription_preprocess,InjuryDescription_preprocess,DamageDescription_preprocess
0,10000,130658,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,left side damages towed by unknown none front...,Moderate,Field Ops 1,Other party at fault,Collision with motor vehicle,2015-03-12 09:05:17.9100000,2015-04-29 11:25:18.3190000,left side damages towed by unknown none front...,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,left side damages towed by unknown none front...
1,10001,130659,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Field Ops 1,Other party at fault,Collision while merging / lane change,2015-03-12 11:46:23.1590000,2015-07-06 08:26:47.5350000,front right headlight front side of bumper dr...,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...
2,10002,130660,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Field Ops 1,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,unknown damages rear bumper trunk right rear ...,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...
3,10003,130661,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Field Ops 1,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,unknown damages rear bumper trunk right rear ...,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...
4,10004,130662,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,Collision with motor vehicle,2015-03-12 16:35:17.3570000,2016-01-29 13:54:25.6150000,none none none,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none


In [15]:
claim_level_body = column_to_list_text(general_df, 'ClaimLevelBody')

In [17]:
exposure_level_body = column_to_list_text(general_df, 'ExposureLevelBody')

In [18]:
accident_description = column_to_list_text(general_df, 'AccidentDescription')

In [19]:
injury_description = column_to_list_text(general_df, 'InjuryDescription')

In [20]:
damage_description = column_to_list_text(general_df, 'DamageDescription')

In [45]:
[[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] for doc in damage_description]
# [token for token in word_tokenize(damage_description) if token.isalpha()]

[['left',
  'side',
  'damages',
  'towed',
  'unknown',
  'none',
  'front',
  'left',
  'side',
  'damages',
  'towed',
  'collision',
  'tow',
  'greenville',
  'sc'],
 ['front',
  'right',
  'headlight',
  'front',
  'side',
  'bumper',
  'drivable',
  'rear',
  'left',
  'side',
  'bumper',
  'caved',
  'rear',
  'left',
  'tire',
  'bent',
  'inward',
  'left',
  'door',
  'wrinkled',
  'close',
  'properly',
  'drivable',
  'residence'],
 ['unknown', 'damages', 'rear', 'bumper', 'trunk', 'right', 'rear', 'light'],
 ['unknown', 'damages', 'rear', 'bumper', 'trunk', 'right', 'rear', 'light'],
 ['none', 'none', 'none'],
 ['none', 'none', 'none'],
 ['front',
  'right',
  'side',
  'damaged',
  'head',
  'lights',
  'hood',
  'damaged',
  'drivable',
  'towed',
  'johnson',
  'towing',
  'address',
  'anderson',
  'rd',
  'phone',
  'right',
  'side',
  'door',
  'window',
  'damaged',
  'drivable'],
 ['none'],
 ['damage',
  'towed',
  'workmans',
  'wrecker',
  'service',
  'eden',
